In [1]:
from qiskit import *
import numpy as np
import networkx as nx

# Part 1: Transpiler

In [2]:
def get_native_gates(qc, optimization_level=3):
    return qiskit.transpile(qc, basis_gates=['rz', 'rx', 'rxx', 'ry'], optimization_level=optimization_level)

### example usage

In [3]:
c = qiskit.QuantumCircuit(2)
# Unitary of the controlled-T gate
c.append(qiskit.quantum_info.Operator(np.diag([
    1, 1, 1, np.exp(1j*np.pi/4)
])), [0, 1])

In [4]:
native_c = get_native_gates(c)
native_c.draw()

global phase: π/16
     ┌──────────┐┌───────────┐ ┌────────┐            ┌───────────┐┌───────────┐»
q_0: ┤ RY(-π/2) ├┤0          ├─┤ RZ(-π) ├────────────┤0          ├┤ RX(-7π/8) ├»
     └┬───────┬─┘│  RXX(π/2) │┌┴────────┴┐┌─────────┐│  RXX(π/2) │├───────────┤»
q_1: ─┤ RZ(π) ├──┤1          ├┤ RY(-π/8) ├┤ RX(-2π) ├┤1          ├┤ RZ(-7π/8) ├»
      └───────┘  └───────────┘└──────────┘└─────────┘└───────────┘└───────────┘»
«     ┌──────────┐
«q_0: ┤ RY(-π/2) ├
«     └──────────┘
«q_1: ────────────
«

# Part 2: Mapper

In [5]:
def interaction_graph_from_circuit(circuit):
    '''
    Builds a weighted interaction graph for a given circuit.

    Nodes are qubits
    Edges are weighted by the number of times pairs of qubits interact
    '''
    g = nx.Graph()
    
    for gate in circuit:
        for q in gate[1]:
            g.add_node(q)
            
        for i in range(len(gate[1])):
            for j in range(i):
                q1 = gate[1][i]
                q2 = gate[1][j]
                if q1 != q2:
                    if (q1, q2) not in g.edges:
                        g.add_edge(q1, q2, weight=1)
                    else:
                        g.edges[q1, q2]['weight'] += 1
        
    return g

In [6]:
def metric(adj, mapping, cost_matrix):
    '''
    Takes in weighted adjacency matrix of interaction graph of quantum circuit
    and the mapping from qubit to hardware id number
    and the cost matrix for a two qubit operation
    
    Returns the cost of running this interaction graph with this mapping.
    '''
    begin = 0
    for i,edges in adj:
        for k,v in edges.items():
            if (i in mapping) and (k in mapping):
                begin += v['weight']*cost_matrix[mapping[i]][mapping[k]]
    return begin

In [7]:
def exec_strategy(adj, cost_matrix):
    '''
    This strategy choose the edges with highest weight (most communicative two qubits), and assign them first
    '''
    
    # Dictionary where keys are the index of qiskit qubit objects (type: int)
    # and values are target_hardware node ids (type: int)
    mapping = {}
    
    # weight, index
    weights = [{'weight': sum([q[1]['weight'] for q in list(i[1].items())]), 
                'qubit': i[0]} 
               for i in adj]
    weights_sorted = sorted(weights, key=lambda f: f['weight'])[::-1]

    # choose first one randomly
    opts = list(range(len(cost_matrix)))
    np.random.shuffle(opts)
    pt = opts.pop()
    seen = 0
    mapping[weights_sorted[seen]['qubit']] = pt
    seen += 1

    while len(weights_sorted) > seen:
        metrics = []
        for o in opts:
            test_mapping = mapping.copy()
            test_mapping[weights_sorted[seen]['qubit']] = o
            metrics.append(metric(adj, test_mapping, cost_matrix))

        best_idx = np.argmin(metrics)
        mapping[weights_sorted[seen]['qubit']] = opts[best_idx]
        seen += 1
        opts = opts[:best_idx] + opts[best_idx+1:]
    return mapping

In [8]:
def map_circuit(quantum_circuit:QuantumCircuit, cost_matrix:np.array):
    '''
    Takes an input program given as a qiskit.QuantumCircuit
    and a target hardware, given as a networkx graph.
    Return a dictionary {qubit : hardware_id}

    The target_hardware graph is an undirected graph with nodes
    labeled 0 to n-1 where n is the total number of qubits.
    '''

    ig = interaction_graph_from_circuit(quantum_circuit)
    assert len(ig.nodes) <= len(cost_matrix), 'Not enough qubits in the hardware'
    adj = list(ig.adjacency())

    mappings  = []
    all_metrics = []
    num_trials = 10
    for _ in range(num_trials):
        mapping = exec_strategy(adj, cost_matrix)
        mappings.append(mapping)
        all_metrics.append(metric(adj, mapping, cost_matrix))
    return mappings[np.argmin(all_metrics)]

### example usage

In [9]:
cost = np.array([
    [0, 0.08, 0.97, 0.2], 
    [0.08, 0, 0.4, 0],
    [0, 0.5, 0, 0],
    [0.2, 0.5, 0.30, 0],    
])

In [10]:
# make cost symmetric
cost = (cost + cost.T)/2
print(cost)

[[0.    0.08  0.485 0.2  ]
 [0.08  0.    0.45  0.25 ]
 [0.485 0.45  0.    0.15 ]
 [0.2   0.25  0.15  0.   ]]


In [11]:
map_circuit(c, cost)

{Qubit(QuantumRegister(2, 'q'), 1): 0, Qubit(QuantumRegister(2, 'q'), 0): 1}

# A bigger example

In [12]:
# Implementation of the Cuccaro Quantum Adder from
#   https://github.com/jmbaker94/quantumcircuitbenchmarks
# Design from the paper
#   https://arxiv.org/abs/quant-ph/0410184

# Majority gate
maj_c = QuantumCircuit(3, name='MAJ')
maj_c.cx(2, 1)
maj_c.cx(2, 0)
maj_c.ccx(0, 1, 2)
maj = maj_c.to_gate(label='MAJ')

# 2-CNOT version of UnMajority and Add gate
uma2_c = QuantumCircuit(3, name='UMA2')
uma2_c.toffoli(0, 1, 2)
uma2_c.cx(2, 0)
uma2_c.cx(0, 1)
uma2 = uma2_c.to_gate(label='UMA2')

# 3-CNOT version of UnMajority and Add gate
# (Allows more parallelism in the circuit)
uma3_c = QuantumCircuit(3, name='UMA3')
uma3_c.x(1)
uma3_c.cx(0, 1)
uma3_c.toffoli(0, 1, 2)
uma3_c.x(1)
uma3_c.cx(2, 0)
uma3_c.cx(2, 1)
uma3 = uma3_c.to_gate(label='UMA3')

def cuccaro_adder(c, cin, a, b, cout, uma=uma3):
    c.append(maj, [cin, b[0], a[0]])
    for i in range(1, len(b)):
        c.append(maj, [a[i-1], b[i], a[i]])

    c.cx(a[-1], cout)

    for i in reversed(range(1, len(b))):
        c.append(uma, [a[i-1], b[i], a[i]])
    c.append(uma, [cin, b[0], a[0]])

def generate_adder_circuit(n, uma=uma3):
    if n % 2 != 0:
        raise ValueError('Odd number of qubits')

    qubits = QuantumRegister
    cin = range(1)
    a = range(1, n//2)
    b = range(n//2, n-1)
    cout = range(n-1, n)
    c = QuantumCircuit(n)
        
    cuccaro_adder(c, cin, a, b, cout, uma=uma)
    return c

def decomposed_adder_circuit(n):
    return qiskit.transpile(generate_adder_circuit(n),
                            basis_gates=['cx', 'rx', 'h', 'rz'])

n=4
print('Cuccaro Adder: n=' + str(n))
cuccaro = decomposed_adder_circuit(n)
cuccaro.draw(fold=-1)

Cuccaro Adder: n=4


global phase: π/4
          ┌───┐                                                                              ┌─────────┐                                                                                                                                         ┌─────────┐                     ┌───┐                                 
q_0: ─────┤ X ├────────────────────────■────────────────────────────────■─────────────────■──┤ RZ(π/4) ├──────────────■──────────────────────────────────────────────■─────────────────────■────────────────────────────────■─────────────────■──┤ RZ(π/4) ├──────────────■──────┤ X ├─────────────────────────────────
          └─┬─┘┌───┐┌───┐┌──────────┐┌─┴─┐┌─────────┐┌───┐┌──────────┐┌─┴─┐┌───────────┐  │  ├─────────┤ ┌─────────┐  │                       ┌───┐                  │  ┌───┐┌──────────┐┌─┴─┐┌─────────┐┌───┐┌──────────┐┌─┴─┐┌───────────┐  │  ├─────────┤ ┌─────────┐  │      └─┬─┘                                 
q_1: ──■────■──┤ H ├┤ X ├┤ RZ(-π/4) ├┤ X ├┤ RZ(π/4) ├┤ X ├┤ RZ(-π/4) ├┤ X ├┤ RZ(-5π/4) ├──┼──┤ RX(π/2) ├─┤ RZ(π/2) ├──┼──────────────────■────┤ H ├──────────────────┼──┤ X ├┤ RZ(-π/4) ├┤ X ├┤ RZ(π/4) ├┤ X ├┤ RZ(-π/4) ├┤ X ├┤ RZ(-5π/4) ├──┼──┤ RX(π/2) ├─┤ RZ(π/2) ├──┼────────■────────────────────────────────■──
     ┌─┴─┐     └───┘└─┬─┘└──────────┘└───┘└─────────┘└─┬─┘├─────────┬┘└───┘└───────────┘┌─┴─┐├─────────┴┐└─────────┘┌─┴─┐┌────────────┐  │  ┌─┴───┴─┐┌────────────┐┌─┴─┐└─┬─┘└──────────┘└───┘└─────────┘└─┬─┘├─────────┬┘└───┘└───────────┘┌─┴─┐├─────────┴┐└─────────┘┌─┴─┐┌────────────┐┌───────┐┌────────────┐┌─┴─┐
q_2: ┤ X ├────────────■────────────────────────────────■──┤ RZ(π/4) ├───────────────────┤ X ├┤ RZ(-π/4) ├───────────┤ X ├┤ RZ(2.6376) ├──┼──┤ RX(π) ├┤ RZ(2.6376) ├┤ X ├──■────────────────────────────────■──┤ RZ(π/4) ├───────────────────┤ X ├┤ RZ(-π/4) ├───────────┤ X ├┤ RZ(2.6376) ├┤ RX(π) ├┤ RZ(2.6376) ├┤ X ├
     └───┘                                                └─────────┘                   └───┘└──────────┘           └───┘└────────────┘┌─┴─┐└───────┘└────────────┘└───┘                                      └─────────┘                   └───┘└──────────┘           └───┘└────────────┘└───────┘└────────────┘└───┘
q_3: ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                       └───┘

In [13]:
native_cuccaro = get_native_gates(cuccaro)
native_cuccaro.draw()

global phase: -5π/4
                                           ┌────────────┐┌──────────┐»
q_0: ──────────────────────────────────────┤1           ├┤ RX(-π/2) ├»
     ┌──────────┐┌────────────┐ ┌───────┐  │  RXX(-π/2) │└┬───────┬─┘»
q_1: ┤ RY(-π/2) ├┤0           ├─┤ RX(π) ├──┤0           ├─┤ RZ(π) ├──»
     └──────────┘│  RXX(-π/2) │┌┴───────┴─┐└┬──────────┬┘ └───────┘  »
q_2: ────────────┤1           ├┤ RX(-π/2) ├─┤ RY(-π/2) ├─────────────»
                 └────────────┘└──────────┘ └──────────┘             »
q_3: ────────────────────────────────────────────────────────────────»
                                                                     »
«      ┌──────────┐                         ┌────────────┐ ┌───────┐  »
«q_0: ─┤ RY(-π/2) ├─────────────────────────┤0           ├─┤ RX(π) ├──»
«     ┌┴──────────┴┐┌──────────┐┌──────────┐│  RXX(-π/2) │┌┴───────┴─┐»
«q_1: ┤1           ├┤ RX(-π/2) ├┤ RZ(-π/4) ├┤1           ├┤ RX(-π/2) ├»
«     │  RXX(-π/2) │├──────────┤└──────────┘└────────────┘└──────────┘»
«q_2: ┤0           ├┤ RX(3π/4) ├──────────────────────────────────────»
«     └────────────┘└──────────┘                                      »
«q_3: ────────────────────────────────────────────────────────────────»
«                                                                     »
«                                                      ┌────────────┐»
«q_0: ─────────────────────────────────────────────────┤0           ├»
«     ┌─────────┐┌────────────┐┌──────────┐┌──────────┐│  RXX(-π/2) │»
«q_1: ┤ RZ(π/4) ├┤1           ├┤ RX(-π/2) ├┤ RZ(-π/4) ├┤1           ├»
«     └─────────┘│  RXX(-π/2) │├──────────┤└──────────┘└────────────┘»
«q_2: ───────────┤0           ├┤ RY(-π/2) ├──────────────────────────»
«                └────────────┘└──────────┘                          »
«q_3: ───────────────────────────────────────────────────────────────»
«                                                                    »
«      ┌───────┐             ┌───────────┐┌──────────┐ ┌───────┐  »
«q_0: ─┤ RZ(π) ├─────────────┤0          ├┤ RX(3π/4) ├─┤ RZ(π) ├──»
«     ┌┴───────┴─┐┌─────────┐│           │├──────────┤ └───────┘  »
«q_1: ┤ RZ(-π/2) ├┤ RY(π/2) ├┤  RXX(π/2) ├┤ RZ(-π/4) ├────────────»
«     └──────────┘└─────────┘│           │├─────────┬┘┌──────────┐»
«q_2: ───────────────────────┤1          ├┤ RY(π/4) ├─┤ RZ(3π/2) ├»
«                            └───────────┘└─────────┘ └──────────┘»
«q_3: ────────────────────────────────────────────────────────────»
«                                                                 »
«                                                                              »
«q_0: ─────────────────────────────────────────────────────────────────────────»
«     ┌────────────┐┌─────────┐ ┌───────┐┌────────────┐┌──────────┐┌──────────┐»
«q_1: ┤0           ├┤ RX(π/2) ├─┤ RZ(π) ├┤1           ├┤ RX(-π/2) ├┤ RZ(-π/4) ├»
«     │            │└─────────┘ └───────┘│  RXX(-π/2) │├─────────┬┘└──────────┘»
«q_2: ┤  RXX(-π/2) ├─────────────────────┤0           ├┤ RX(π/2) ├─────────────»
«     │            │┌──────────┐         └────────────┘└─────────┘             »
«q_3: ┤1           ├┤ RX(-π/2) ├───────────────────────────────────────────────»
«     └────────────┘└──────────┘                                               »
«     ┌────────────┐┌──────────┐                                     »
«q_0: ┤0           ├┤ RX(7π/4) ├─────────────────────────────────────»
«     │  RXX(-π/2) │├──────────┤┌─────────┐┌────────────┐┌──────────┐»
«q_1: ┤1           ├┤ RX(-π/2) ├┤ RZ(π/4) ├┤1           ├┤ RX(-π/2) ├»
«     └────────────┘└──────────┘└─────────┘│  RXX(-π/2) │├─────────┬┘»
«q_2: ─────────────────────────────────────┤0           ├┤ RY(π/2) ├─»
«                                          └────────────┘└─────────┘ »
«q_3: ───────────────────────────────────────────────────────────────»
«                                                                    »
«                 ┌────────────┐            ┌────────────┐            »
«q

In [14]:
cost = np.array([
    [0, 0.08, 0.97, 0.2], 
    [0.08, 0, 0.4, 0],
    [0, 0.5, 0, 0],
    [0.2, 0.5, 0.30, 0],    
])
# make cost symmetric
cost = (cost + cost.T)/2
print(cost)

[[0.    0.08  0.485 0.2  ]
 [0.08  0.    0.45  0.25 ]
 [0.485 0.45  0.    0.15 ]
 [0.2   0.25  0.15  0.   ]]


In [15]:
map_circuit(native_cuccaro, cost)

{Qubit(QuantumRegister(4, 'q'), 1): 0,
 Qubit(QuantumRegister(4, 'q'), 0): 1,
 Qubit(QuantumRegister(4, 'q'), 2): 3,
 Qubit(QuantumRegister(4, 'q'), 3): 2}